# INF8245E: Machine Learning | Assignment #3

**Louis Plessis (1933334)** | 14 November 2021

# 1. Data pre-processing

In [1]:
import pandas as pd
import numpy as np

strings = pd.read_csv("medical_dataset/train.csv", header=None).to_numpy()
strings = np.delete(strings, 0, 0)
strings = np.delete(strings, 0, 1).flatten()

import string
trans = str.maketrans("", "", string.punctuation)

vocab = []

#Removing punctuation and lower-casing words
for i in range(len(strings)):
    strings[i] = strings[i].translate(trans)
    strings[i] = strings[i].lower()
    
    for j in strings[i].split():
        if j not in vocab:
            vocab.append(j)

In [2]:
#Running this will take a long time

frequency = []

for i in vocab:
    count = 0
    for string in strings:
        count += string.count(i)
    frequency.append(count)

In [10]:
import csv

freqsort = np.argsort(frequency)

#Writing vocabulary file
with open('medical_text-vocab.txt', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for i in range(10000):
        row = str(vocab[freqsort[len(freqsort)-1-i]]) + "\t" + str(i+1) + "\t" + str(frequency[freqsort[len(freqsort)-1-i]])
        writer.writerow([row])

In [69]:
import pandas as pd
import numpy as np

vocabulary = []

#Reading vocabulary file
with open('medical_text-vocab.txt') as file:
    reader = csv.reader(file, delimiter = "\t")
    for row in reader:
        vocabulary.append(row)

voc = np.delete(vocabulary, 2, 1)
voc = np.delete(voc, 1, 1).flatten()
voc = list(voc)

In [70]:
train = pd.read_csv("medical_dataset/train.csv", header=None).to_numpy()
test = pd.read_csv("medical_dataset/test.csv", header=None).to_numpy()
valid = pd.read_csv("medical_dataset/valid.csv", header=None).to_numpy()

train = np.delete(train, 0, 0)
test = np.delete(test, 0, 0)
valid = np.delete(valid, 0, 0)

import string
trans = str.maketrans("", "", string.punctuation)

#Removing punctuation and lower-casing words
for i in range(len(train)):
    train[i][1] = train[i][1].translate(trans)
    train[i][1] = train[i][1].lower()
    
for i in range(len(test)):
    test[i][1] = test[i][1].translate(trans)
    test[i][1] = test[i][1].lower()
    
for i in range(len(valid)):
    valid[i][1] = valid[i][1].translate(trans)
    valid[i][1] = valid[i][1].lower()

In [71]:
#Writing training file
with open('medical_text-train.txt', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for i in range(len(train)):
        row = ""
        for word in train[i][1].split():
            try:
                word_index = str(voc.index(word) + 1) + " "
            except:
                word_index = ""
            row += word_index
        row = row + "\t" + str(train[i][0])
        writer.writerow([row])

In [72]:
#Writing testing file
with open('medical_text-test.txt', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for i in range(len(test)):
        row = ""
        for word in test[i][1].split():
            try:
                word_index = str(voc.index(word) + 1) + " "
            except:
                word_index = ""
            row += word_index
        row = row + "\t" + str(test[i][0])
        writer.writerow([row])

In [73]:
#Writing validation file
with open('medical_text-valid.txt', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for i in range(len(valid)):
        row = ""
        for word in valid[i][1].split():
            try:
                word_index = str(voc.index(word) + 1) + " "
            except:
                word_index = ""
            row += word_index
        row = row + "\t" + str(valid[i][0])
        writer.writerow([row])

# 2. Binary bag-of-words (BBoW)

In [37]:
import pandas as pd
import numpy as np
import csv

#Reading training file
train = []
with open('medical_text-train.txt') as file:
    reader = csv.reader(file, delimiter = "\t")
    for row in reader:
        train.append(row)
        
#Reading testing file
test = []
with open('medical_text-test.txt') as file:
    reader = csv.reader(file, delimiter = "\t")
    for row in reader:
        test.append(row)
        
#Reading validation file
valid = []
with open('medical_text-valid.txt') as file:
    reader = csv.reader(file, delimiter = "\t")
    for row in reader:
        valid.append(row)

In [38]:
#Binary Bag of Words Representation

x_train = []
y_train = []
for i in range(len(train)):
    a = []
    words = train[i][0].split()
    for j in range(10000):
        if str(j+1) in words:
            a.append(1)
        else:
            a.append(0)
    x_train.append(a)
    y_train.append(train[i][1])
    
x_test = []
y_test = []
for i in range(len(test)):
    a = []
    words = test[i][0].split()
    for j in range(10000):
        if str(j+1) in words:
            a.append(1)
        else:
            a.append(0)
    x_test.append(a)
    y_test.append(test[i][1])
    
x_valid = []
y_valid = []
for i in range(len(valid)):
    a = []
    words = valid[i][0].split()
    for j in range(10000):
        if str(j+1) in words:
            a.append(1)
        else:
            a.append(0)
    x_valid.append(a)
    y_valid.append(valid[i][1])

In [80]:
save_bbow_x_train = x_train
save_bbow_x_valid = x_valid
save_bbow_x_test = x_test

In [72]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import f1_score

def computeF1ScoreA(classifier):
    classifier = classifier.fit(x_train, y_train)
    
    predictions_train = classifier.predict(x_train)
    predictions_valid = classifier.predict(x_valid)
    predictions_test = classifier.predict(x_test)

    train_score = f1_score(y_train, predictions_train, average="macro")
    valid_score = f1_score(y_valid, predictions_valid, average="macro")
    test_score = f1_score(y_test, predictions_test, average="macro")
    
    print("************ F1-Scores ************")
    print("Training:\t", train_score)
    print("Validation:\t", valid_score)
    print("Testing:\t", test_score)

### Random Classifier

In [73]:
dummy_classifier = DummyClassifier(strategy="uniform")
computeF1ScoreA(dummy_classifier)

************ F1-Scores ************
Training:	 0.25318763066226246
Validation:	 0.23045385773124563
Testing:	 0.257883563203843


### Majority-Class Classifier

In [74]:
majority_class_classifier = DummyClassifier(strategy="most_frequent")
computeF1ScoreA(majority_class_classifier)

************ F1-Scores ************
Training:	 0.120996778472617
Validation:	 0.12424698795180723
Testing:	 0.14183381088825217


In [75]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import f1_score
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

def computeF1ScoreB(classifier, parameters):
    
    classifier = GridSearchCV(classifier, parameters, refit=True)
    classifier = classifier.fit(x_train, y_train)
    
    predictions_train = classifier.predict(x_train)
    predictions_valid = classifier.predict(x_valid)
    predictions_test = classifier.predict(x_test)

    train_score = f1_score(y_train, predictions_train, average="macro")
    valid_score = f1_score(y_valid, predictions_valid, average="macro")
    test_score = f1_score(y_test, predictions_test, average="macro")
    
    print("************ F1-Scores ************")
    print("Training:\t", train_score)
    print("Validation:\t", valid_score)
    print("Testing:\t", test_score)
    print("\n*** Best hyper-parameter values ***")
    print(classifier.best_params_)

### Naive Bayes

In [76]:
computeF1ScoreB(BernoulliNB(), [{'alpha': np.arange(0.1,1,0.1)}])

************ F1-Scores ************
Training:	 0.5243230447997342
Validation:	 0.4453180264457025
Testing:	 0.4672239541944292

*** Best hyper-parameter values ***
{'alpha': 0.6}


### Decision Trees

In [77]:
computeF1ScoreB(DecisionTreeClassifier(), [{'max_depth': [1, 2, 5, 10]}])

************ F1-Scores ************
Training:	 0.6538501176678703
Validation:	 0.5394842997924081
Testing:	 0.5913772738010867

*** Best hyper-parameter values ***
{'max_depth': 10}


### Logistic Regression

In [78]:
computeF1ScoreB(LogisticRegression(), [{'C': [0.01, 0.1, 1, 10, 100]}])

C:\Users\louis\.julia\conda\3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\louis\.julia\conda\3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

C:\Users\louis\.julia\conda\3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\louis\.julia\conda\3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

************ F1-Scores ************
Training:	 0.8226849082758394
Validation:	 0.44471145768048514
Testing:	 0.4933905041682139

*** Best hyper-parameter values ***
{'C': 1}


### Linear SVM/

In [79]:
computeF1ScoreB(LinearSVC(), [{'C': [0.01, 0.1, 1, 10, 100]}])

C:\Users\louis\.julia\conda\3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\louis\.julia\conda\3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\louis\.julia\conda\3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\louis\.julia\conda\3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\louis\.julia\conda\3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\louis\.julia\conda\3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to 

************ F1-Scores ************
Training:	 0.8223413846361506
Validation:	 0.5216290554386827
Testing:	 0.5382102261041218

*** Best hyper-parameter values ***
{'C': 1}


# 3. Frequency bag-of-words (FBoW)

In [44]:
# Frequency Bag of Words Representation

fbow_x_train = []
for i in range(len(train)):
    a = []
    words = train[i][0].split()
    for j in range(10000):
        a.append(words.count(str(j+1)))
    a = np.array(a)
    a = a/np.sum(a)
    fbow_x_train.append(a)

In [45]:
fbow_x_test = []
for i in range(len(test)):
    a = []
    words = test[i][0].split()
    for j in range(10000):
        a.append(words.count(str(j+1)))
    a = np.array(a)
    a = a/np.sum(a)
    fbow_x_test.append(a)
    
fbow_x_valid = []
for i in range(len(valid)):
    a = []
    words = valid[i][0].split()
    for j in range(10000):
        a.append(words.count(str(j+1)))
    a = np.array(a)
    a = a/np.sum(a)
    fbow_x_valid.append(a)

In [58]:
save_fbow_x_train = fbow_x_train
save_fbow_x_test = fbow_x_test
save_fbow_x_valid = fbow_x_valid

### Naive Bayes

In [81]:
x_train = save_fbow_x_train
x_test = save_fbow_x_test
x_valid = save_fbow_x_valid

computeF1ScoreB(BernoulliNB(), [{'alpha': np.arange(0.1,1,0.1)}])

************ F1-Scores ************
Training:	 0.5243230447997342
Validation:	 0.4453180264457025
Testing:	 0.4672239541944292

*** Best hyper-parameter values ***
{'alpha': 0.6}


### Decision Trees

In [82]:
computeF1ScoreB(DecisionTreeClassifier(), [{'max_depth': [1, 2, 5, 10]}])

************ F1-Scores ************
Training:	 0.6649919768970892
Validation:	 0.5753923197816475
Testing:	 0.5863910797466454

*** Best hyper-parameter values ***
{'max_depth': 10}


### Logistic Regression

In [83]:
computeF1ScoreB(LogisticRegression(), [{'C': [0.01, 0.1, 1, 10, 100]}])

C:\Users\louis\.julia\conda\3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\louis\.julia\conda\3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

************ F1-Scores ************
Training:	 0.45926741730734605
Validation:	 0.4179133177475002
Testing:	 0.4205989741981914

*** Best hyper-parameter values ***
{'C': 10}


### Linear SVM/

In [84]:
computeF1ScoreB(LinearSVC(), [{'C': [0.01, 0.1, 1, 10], 'max_iter': np.arange(100,1000,100)}])

C:\Users\louis\.julia\conda\3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\louis\.julia\conda\3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\louis\.julia\conda\3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\louis\.julia\conda\3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\louis\.julia\conda\3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\louis\.julia\conda\3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to 

************ F1-Scores ************
Training:	 0.39952614300221356
Validation:	 0.39011900287444246
Testing:	 0.3831947217586665

*** Best hyper-parameter values ***
{'C': 1, 'max_iter': 100}
